<a href="https://colab.research.google.com/github/RohanGforwork/ML_text_summarization/blob/main/team_1_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [4]:
!pip install datasets

In [5]:
from datasets import load_dataset

df = load_dataset("agentlans/wikipedia-paragraph-summaries")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
from datasets import load_dataset
import pandas as pd

# Load dataset
dataset = load_dataset("agentlans/wikipedia-paragraph-summaries")

# Convert to Pandas DataFrame (for 'train' split)
df = dataset["train"].to_pandas()

# Show first few rows
print(df.head())


   id                                              input  \
0   1  Perhaps the first public statement on the matt...   
1   2  Over the next several decades, the death penal...   
2   3  Chester Himes was born in Jefferson City, Miss...   
3   4  The Euallomyces life cycle is an anisogamous a...   
4   5  Humenik joined the PGA Tour in 1989, gaining h...   

                                              output  
0  Catherine II's 1767 statement and later events...  
1  The Soviet death penalty had a complex history...  
2  Chester Himes, born in Missouri in 1909, grew ...  
3  The Euallomyces life cycle alternates between ...  
4  Humenik began his PGA Tour career in 1989, str...  


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21811 entries, 0 to 21810
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      21811 non-null  int64 
 1   input   21811 non-null  object
 2   output  21811 non-null  object
dtypes: int64(1), object(2)
memory usage: 511.3+ KB


In [8]:
df.isnull().sum()

,0
id,0
input,0
output,0


In [9]:

df = df['input'].to_frame()

In [10]:
print(df.head())

                                               input
0  Perhaps the first public statement on the matt...
1  Over the next several decades, the death penal...
2  Chester Himes was born in Jefferson City, Miss...
3  The Euallomyces life cycle is an anisogamous a...
4  Humenik joined the PGA Tour in 1989, gaining h...


In [11]:
import pandas as pd
import spacy
import re
import string
from datasets import load_dataset
from sklearn.model_selection import train_test_split

# Load dataset from Hugging Face
dataset = load_dataset("agentlans/wikipedia-paragraph-summaries")

# Convert to Pandas DataFrame
df = dataset["train"].to_pandas()

# Load English NLP model (disable unnecessary components for speed)
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# Precompile regex and translation table for faster processing
url_pattern = re.compile(r"http\S+")
translator = str.maketrans("", "", string.punctuation)

def clean_text(text):
    """Basic text cleaning: Lowercase, remove URLs & punctuation"""
    text = text.lower()
    text = url_pattern.sub("", text)  # Remove URLs
    text = text.translate(translator)  # Remove punctuation
    return text

# Apply initial cleaning to paragraphs & summaries
df["cleaned_paragraph"] = df["input"].apply(clean_text)
df["cleaned_summary"] = df["output"].apply(clean_text)

def process_texts(texts):
    """Lemmatization & stopword removal using spaCy (batch processing)"""
    docs = list(nlp.pipe(texts, batch_size=512))  # Adjust batch_size based on RAM
    return [" ".join([token.lemma_ for token in doc if not token.is_stop]) for doc in docs]

# Apply NLP processing in batches
df["cleaned_paragraph"] = process_texts(df["cleaned_paragraph"])
df["cleaned_summary"] = process_texts(df["cleaned_summary"])

# Train-Test Split
train_paragraphs, test_paragraphs, train_summaries, test_summaries = train_test_split(
    df["cleaned_paragraph"], df["cleaned_summary"], test_size=0.2, random_state=42
)

print("✅ Preprocessing Completed!")


✅ Preprocessing Completed!


In [12]:
from sklearn.model_selection import train_test_split

# Ensure dataframe has no missing values
df = df.dropna(subset=["cleaned_paragraph", "cleaned_summary"])

# Train-Test Split
train_paragraphs, test_paragraphs, train_summaries, test_summaries = train_test_split(
    df["cleaned_paragraph"].tolist(),
    df["cleaned_summary"].tolist(),
    test_size=0.2,
    random_state=42
)

# Convert lists to dataset format
train_data = {"paragraph": train_paragraphs, "summary": train_summaries}
test_data = {"paragraph": test_paragraphs, "summary": test_summaries}

print(f"✅ Training Samples: {len(train_paragraphs)}, Testing Samples: {len(test_paragraphs)}")


✅ Training Samples: 17448, Testing Samples: 4363


In [22]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# ✅ Load tokenizer & model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")


In [13]:
!pip install transformers datasets accelerate


In [14]:
from transformers import T5Tokenizer

# Load T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")  # Use "t5-base" for a larger model

# Tokenization function
def preprocess_data(examples):
    model_input = tokenizer(
        examples["paragraph"],
        max_length=512,  # Truncate long paragraphs
        padding="max_length",
        truncation=True,
    )

    labels = tokenizer(
        examples["summary"],
        max_length=150,  # Set max length for summary
        padding="max_length",
        truncation=True,
    )

    model_input["labels"] = labels["input_ids"]
    return model_input

# Convert train/test data to Hugging Face dataset
from datasets import Dataset

train_dataset = Dataset.from_dict(train_data).map(preprocess_data, batched=True)
test_dataset = Dataset.from_dict(test_data).map(preprocess_data, batched=True)

print(f"✅ Tokenization Completed! {len(train_dataset)} training samples.")


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/17448 [00:00<?, ? examples/s]

Map:   0%|          | 0/4363 [00:00<?, ? examples/s]

✅ Tokenization Completed! 17448 training samples.


In [15]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [17]:
from transformers import TrainingArguments, Trainer


In [18]:
!pip install accelerate


In [20]:
!accelerate launch train_script.py


The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `0`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
/usr/bin/python3: can't open file '/content/train_script.py': [Errno 2] No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/accelerate", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/accelerate/commands/accelerate_cli.py", line 48, in main
    args.func(args)
  File "/usr/local/lib/python3.11/dist-packages/accelerate/commands/launch.py", line 1172, in launch_command
    simple_launcher(args)
  File "/usr/local/lib/python3.11/dist-packages/accelerate/commands/launch.py", line 762, in simple_launcher
    raise subproc

In [ ]:
training_args = TrainingArguments(
    output_dir="./t5_summarization",
    evaluation_strategy="epoch",
    learning_rate=3e-5,  # 🔥 Slightly increased learning rate for faster convergence
    per_device_train_batch_size=16,  # 🔥 Increase batch size (if GPU memory allows)
    per_device_eval_batch_size=16,
    num_train_epochs=2,  # 🔥 Reduce epochs slightly
    weight_decay=0.01,
    save_total_limit=1,  # 🔥 Keep only the latest model checkpoint
    logging_dir="./logs",
    logging_steps=500,  # 🔥 Reduce logging frequency
    fp16=True,  # 🔥 Enable mixed precision for faster training
    gradient_accumulation_steps=2,  # 🔥 Effective batch size increase without high memory usage
    optim="adamw_torch",  # 🔥 Faster optimizer (if supported)
    dataloader_pin_memory=True,  # 🔥 Speed up memory access
)

# ✅ Trainer Object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# ✅ Start Training
trainer.train()


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


In [16]:
# ✅ Training Arguments (Optimized for Speed)
training_args = TrainingArguments(
    output_dir="./t5_summarization",
    eval_strategy="epoch",
    learning_rate=3e-5,  # 🔥 Slightly higher learning rate for faster convergence
    per_device_train_batch_size=16,  # 🔥 Increase batch size (if GPU memory allows)
    per_device_eval_batch_size=16,
    num_train_epochs=2,  # 🔥 Reduce epochs
    weight_decay=0.01,
    save_total_limit=1,  # 🔥 Keep only the latest model checkpoint
    push_to_hub=False,
    logging_dir="./logs",
    fp16=True,  # 🔥 Enable mixed precision for speedup
)

# ✅ Trainer Object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# ✅ Start Training
trainer.train()


NameError: name 'TrainingArguments' is not defined

In [ ]:
model.save_pretrained("./t5_summarizer")
tokenizer.save_pretrained("./t5_summarizer")


In [ ]:
def summarize_text(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs["input_ids"], max_length=150, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Test summarization
sample_text = "Your paragraph here..."
print("Summary:", summarize_text(sample_text))


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset

# Load BART model & tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Tokenization function
def tokenize_function(examples):
    inputs = tokenizer(examples["cleaned_paragraph"], max_length=1024, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["cleaned_summary"], max_length=256, truncation=True, padding="max_length")

    inputs["labels"] = labels["input_ids"]
    return inputs

# Convert to Hugging Face Dataset
train_data = Dataset.from_pandas(df.iloc[:int(0.8 * len(train_summaries))])  # 80% Train
test_data = Dataset.from_pandas(df.iloc[int(0.8 * len(df)):])  # 20% Test

# Tokenize dataset
train_data = train_data.map(tokenize_function, batched=True)
test_data = test_data.map(tokenize_function, batched=True)

# Training setup
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    save_steps=500,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    report_to="none",
    fp16=True,  # Use Mixed Precision for TPU
    gradient_accumulation_steps=4,  # Accumulate gradients for larger effective batch
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
)

# Train the model
trainer.train()


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/long-t5-local-base")

# Tokenize training and testing data
train_encodings = tokenizer(list(train_summaries), padding=True, truncation=True, max_length=512, return_tensors="pt")
test_encodings = tokenizer(list(test_summaries), padding=True, truncation=True, max_length=512, return_tensors="pt")


In [ ]:
"""# long t5 conditional generation
from transformers import LongT5ForConditionalGeneration, Trainer, TrainingArguments
import torch

model = LongT5ForConditionalGeneration.from_pretrained("google/long-t5-local-base")

# Convert data to PyTorch tensors
train_labels = tokenizer(list(train_summaries), padding=True, truncation=True, max_length=150, return_tensors="pt").input_ids
test_labels = tokenizer(list(test_summaries), padding=True, truncation=True, max_length=150, return_tensors="pt").input_ids

train_data = [{"input_ids": train_encodings.input_ids[i], "attention_mask": train_encodings.attention_mask[i], "labels": train_labels[i]} for i in range(len(train_summaries))]
test_data = [{"input_ids": test_encodings.input_ids[i], "attention_mask": test_encodings.attention_mask[i], "labels": test_labels[i]} for i in range(len(test_summaries))]

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
)

# Start Training
trainer.train()"""


In [ ]:
model.save_pretrained("./fine_tuned_longt5")
tokenizer.save_pretrained("./fine_tuned_longt5")


In [ ]:
def summarize_tweet(tweet, model, tokenizer):
    input_ids = tokenizer.encode("summarize: " + tweet, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(input_ids, max_length=150, num_beams=3, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

test_tweet = "Your sample social media text here..."
print("Summary:", summarize_tweet(test_tweet, model, tokenizer))


In [ ]:
import spacy
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load Summarization Models
pegasus_model_name = "google/pegasus-xsum"
longt5_model_name = "google/long-t5-tglobal-base"

pegasus_tokenizer = AutoTokenizer.from_pretrained(pegasus_model_name)
pegasus_model = AutoModelForSeq2SeqLM.from_pretrained(pegasus_model_name)

longt5_tokenizer = AutoTokenizer.from_pretrained(longt5_model_name)
longt5_model = AutoModelForSeq2SeqLM.from_pretrained(longt5_model_name)

# Function to clean text using spaCy
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = text.translate(str.maketrans("", "", string.punctuation))  # Remove punctuation

    doc = nlp(text)  # Process text with spaCy
    cleaned_text = " ".join([token.lemma_ for token in doc if not token.is_stop])  # Lemmatize & remove stopwords
    return cleaned_text

# Load dataset


# Apply text cleaning
df['cleaned_text'] = df['text'].apply(clean_text)

# Convert text to TF-IDF vectors
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Limit vocabulary size for efficiency
tfidf_matrix = tfidf_vectorizer.fit_transform(df['cleaned_text'])

# Function to summarize text dynamically
def summarize_text(text):
    text_length = len(text)

    # Select model dynamically
    if text_length <= 300:
        tokenizer, model = pegasus_tokenizer, pegasus_model
        max_len = 128  # Shorter summary for tweets
    else:
        tokenizer, model = longt5_tokenizer, longt5_model
        max_len = 200  # Longer summary for paragraphs

    # Tokenize input
    input_ids = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)

    # Generate summary
    summary_ids = model.generate(input_ids, max_length=max_len, num_beams=5, early_stopping=True)

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Apply summarization function
df['summary'] = df['cleaned_text'].apply(summarize_text)

# Display results
print(df[['text', 'summary']].head())


In [ ]:
df['text'][142592]

In [ ]:
df

In [ ]:
'''!pip install transformers[torch]
!pip install accelerate -U'''

In [ ]:
import os

In [ ]:
from transformers import LongT5ForConditionalGeneration, AutoTokenizer

model = LongT5ForConditionalGeneration.from_pretrained("Stancld/longt5-tglobal-large-16384-pubmed-3k_steps")
tokenizer = AutoTokenizer.from_pretrained("Stancld/longt5-tglobal-large-16384-pubmed-3k_steps")

def summarize_tweet(tweet, model=model, tokenizer=tokenizer):
    # Preprocess and tokenize the tweet
    input_ids = tokenizer.encode("summarize: " + tweet, return_tensors="pt", max_length=1000, truncation=True)

    # Generate summary using the LongT5 model
    summary_ids = model.generate(input_ids, max_length=512, num_beams=1, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

tweet = input("Enter a tweet to summarize: ")
summary = summarize_tweet(tweet)

print("Original tweet:", tweet)
print("Summary:", summary)